# Deploy Machine Learning Job on Truefoundry
This notebook demonstrates a demo on how you can deploy an image classification model trained on mnist dataset as a Gradio App on truefoundry platform.

## Prerequisites

Before we begin, make sure you have the following prerequisites in place:

1. **Install `servicefoundry`** (Note: `servicefoundry` is pre-installed in Truefoundry notebooks). You can install it using the following command:

In [ ]:
!pip install -U "servicefoundry"

2. **Login to servicefoundry**

Enter your host in the `--host` argument, eg: "https://your-domain.truefoundry.com"

In [ ]:
!sfy login --host "<ENTER YOUR HOST HERE>"

3. **Select the `Workspace`** in which you want to deploy your application. <br>Once you run the cell below you will get a prompt to enter your workspace. <br>
    * **Step 1:** Navigate to the **Workspace** tab on the left panel of your User Interface.
    * **Step 2:** Identify the Workspace you want to deploy the application in.
    * **Step 3:** Copy the Workspace FQN <br>
    ![Copying Workspace FQN](https://files.readme.io/730fee2-Screenshot_2023-02-28_at_2.08.34_PM.png)
    * **Step 4:** Paste the  Workspace FQN in the prompt and press enter.

In [ ]:
workspace_fqn = input("Enter your Workspace FQN: ")

4. **Setup Logging**

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Clone the Getting Started Repo

In this step, we will clone the Truefoundry Getting Started repository. This repository contains the job code that we are going to deploy.

In [ ]:
!git clone https://github.com/truefoundry/getting-started-examples.git

Now let's `cd` into the directory containing our inference code, i.e `getting-started-examples/customer-churn`

In [ ]:
%cd getting-started-examples/customer-churn

### Code Structure
Before we proceed, let's take a quick look at the structure of the code you'll be deploying:

```
.
|_ app.py: Contains the Gradio Service code used to serve your model.
|_ requirements.txt: Dependency file.
|_ gen_example_images.py: Code to generate example images that you can use to test your Gradio service.
|_ train.py: Contains the Training code used to train the model we are deploying.
```

The `app.py` file houses the code that enables you to deploy and interact with your trained model using Gradio. Here's a brief overview of its components:

* **Prediction Function:**
    * The predict function is responsible for making predictions using the trained model.
    * Preprocess the input image and prepare it for model inference.
    * Load the trained model and perform predictions to identify the digit in the image.
* **Gradio Interface Setup:**
    * Create a Gradio interface that takes an input image and produces a predicted label.
    * Provide examples of images ("0.jpg" and "1.jpg") for users to easily test the interface.

```python
import gradio as gr  
import tensorflow as tf  
import numpy as np  
from PIL import Image

def predict(img_arr):
	# Preprocess the image before passing it to the model
  img_arr = tf.expand_dims(img_arr, 0)  
  img_arr = img_arr[:, :, :, 0]  # Keep only the first channel (grayscale)

	# Load the trained model
  loaded_model = tf.keras.models.load_model('mnist_model.h5')

	# Make predictions
  predictions = loaded_model.predict(img_arr)  
  predicted_label = tf.argmax(predictions[0]).numpy()

  return str(predicted_label)

# Setup the gradio interface
gr.Interface(fn=predict,  
             inputs="image",  
             outputs="label",  
             examples=[["0.jpg"], ["1.jpg"]]  
).launch(server_name="0.0.0.0", server_port=8080)
```




## Deploying Your Machine Learning Job

Now, let's move on to the deployment steps.

### Step 1: Set Up Deployment Configuration
In this step, you will define your deployment configuration using the ServiceFoundry CLI. We will provide explanations for each configuration, and afterwards we will bring all of this together into a `servicefoundry.yaml` file.<br>

For deploying via ServiceFoundry CLI you need to create a `servicefoundry.yaml` file with a YAML configuration of your Deployment

#### Name
In the provided Python script, set a unique identifier for your job using the name field.

```yaml
name: churn-prediction-job
```

#### Image

* Choosing the Right Approach for specifying image:
    Depending on your scenario, you can choose to deploy either a pre-built Docker image or build a Docker image from your source code.
    
* Using Pre-Built Images
    If you already have a Docker image that you've previously built and pushed to a container registry, you can use `type: image`.
    The `type: image` would simply reference the pre-built image URL and use it for deployment.
* Using Build for Source Code
    In cases where you don't have a pre-built image, you'll use the Build option to create an image from your source code.
    This scenario applies when you want to package and deploy your application from scratch.
    * Creating DockerFile with PythonBuild
        If you don't have a Dockerfile but your application is written in Python, you can use the `type: tfy-python-buildpack`.
        The PythonBuild class will inspect your Python code and create a Dockerfile automatically based on the code's requirements.
    * Choosing DockerBuild for Dockerfile
        If you have a pre-existing Dockerfile, you can use the `type: dockerfile`.
        This allows you to directly reference the Dockerfile present in your code repository.

In this case given we did not have a prebuilt image, and no dockerfile in our source code we are using `tfy-python-buildpack`, which takes our code configuration from us and templatizes a Dockerfile for us.

```yaml
image:
  type: build
  build_spec:
    type: tfy-python-buildpack
    command: python main.py --n_neighbors {{n_neighbors}} --weights {{weights}}
    python_version: '3.9'
    requirements_path: requirements.txt
    build_context_path: ./
  build_source:
    type: local
```

#### Params
The `params:` key empowers you to configure hyperparameters and pass them to create distinct job runs.

For each parameter, provide the following details:

- **Name:** Enter a descriptive name for the parameter.
- **Default value:** Specify the default value for the parameter.
- **Description:** Include a brief description of the parameter's purpose.
- **Param type:** Can be either string or an ML Repo

Note that the name of Param are same as what we filled in the comman's {{}} template. `python main.py --n_neighbors {{n_neighbors}} --weights {{weights}`

```yaml
params:
  - name: n_neighbors
    default: '5'
    param_type: string
    description: 'Number of neighbors to use by default'
  - name: weights
    default: uniform
    param_type: string
    description: 'Weight function used in prediction. Possible values: uniform, distance'
```

#### Resources
Allocate computing resources (CPU, memory, storage) for your job using the Resources option.<br>
* **CPU** refers to the computing power available to your application
* **Memory** refers to how much space your application has to hold and work with data while it's running
* **Ephemeral storage** is where your application can temporarily store files and data

Requests and Limits:

* **Request** is like asking for a certain amount of a resource. It's what your application initially asks for to start working properly.
* **Limit** is like setting a maximum value. It restricts how much of a resource (like CPU or memory) your application can use.

So for each category of resource you specify the Request and Limits

```yaml
resources:
  cpu_limit: 0.3
  gpu_count: 0
  cpu_request: 0.3
  memory_limit: 500
  memory_request: 500
  ephemeral_storage_limit: 600
  ephemeral_storage_request: 600
```


### Step 2: Bring all of the configuration together via the Job Class and Deploy

To deploy your machine learning job, you need to create a `servicefoundry.yaml` file with the configuration library. This will encapsulate all the necessary configurations and parameters for deploying and managing your job.

In [ ]:
%%writefile servicefoundry.yaml
name: churn-prediction-job
type: job
image:
  type: build
  build_spec:
    type: tfy-python-buildpack
    command: python main.py --n_neighbors {{n_neighbors}} --weights {{weights}}
    python_version: '3.9'
    requirements_path: requirements.txt
    build_context_path: ./
  build_source:
    type: local
params:
  - name: n_neighbors
    default: '5'
    param_type: string
    description: Number of neighbors to use by default
  - name: weights
    default: uniform
    param_type: string
    description: 'Weight function used in prediction. Possible values: uniform, distance'
resources:
  cpu_limit: 0.3
  cpu_request: 0.3
  memory_limit: 500
  memory_request: 500
  ephemeral_storage_limit: 600
  ephemeral_storage_request: 600

After configuring your deployment settings, you can deploy the job using the deploy method. Here we are replacing the WORKSPACE_FQN with the workspace_fqn we stored earlier.

In [ ]:
# Deploy the job
!servicefoundry deploy --workspace-fqn=YOUR_WORKSPACE_FQN

Once the build is complete, you will see a link to the dashboard after a message like `You can find the application on the dashboard:-`. <br>Click on the link to access the deployment dashboard.

# Effortless Hyperparameter Experimentation

Once your deployment is active, navigate to your specific job by clicking on it. This action will open a dedicated dashboard displaying various job details, including the **Run Job** button.

![](https://files.readme.io/cfff7cd-Screenshot_2023-08-23_at_1.48.02_PM.png)

Clicking this button will trigger a modal to appear:

![](https://files.readme.io/971a7fe-Screenshot_2023-08-23_at_1.51.38_PM.png)

Within this modal, you can effortlessly adjust hyperparameter values for rapid experimentation.

After configuring the modal, submit it using the Run Job button. This action will redirect you to the Job Runs tab. Within a few moments, your job status should switch to Finished.

Proceed by clicking on the logs button to access your job's results:

![](https://files.readme.io/1b79056-Screenshot_2023-08-28_at_7.17.03_AM.png)

Now closing, clicking the purple **churn-train-job** badge will grant you access to the Key Metrics, Hyperparameters, Logged Model, and Associated Artifacts from the run.


![](https://files.readme.io/0113700-Screenshot_2023-08-28_at_7.14.36_AM.png)

# Additional Capabilities of Jobs

Let's delve into the advanced functionalities that Jobs offer, extending beyond deployment strategies:

- **Continuous Integration/Continuous Deployment (CI/CD) via Truefoundry:** Integrate Jobs with Truefoundry for streamlined CI/CD pipelines, ensuring efficient code integration, testing, and deployment.
- **Cron Jobs:** Schedule Jobs to run at specified intervals using cron-like expressions, automating recurring tasks and processes.
- **Job Parametrization:** Configure Jobs with parameters, allowing you to customize execution by providing dynamic input values.
- **Programmatic Job Triggers:** Trigger Jobs programmatically via APIs, enabling seamless automation and integration with external systems.
- **Additional Configurations:** Access a range of supplementary configurations to fine-tune job behavior and optimize performance.